In [1]:
import sys 
sys.path.append('..')

In [2]:
import os
from src.datasets.mimic_cxr_dataset import MIMICCXRDataModule
from pytorch_lightning import Trainer, seed_everything
import torchvision.transforms as T
# sets seeds for numpy, torch, python.random and PYTHONHASHSEED.
seed_everything(42)


/Users/caghankoksal/miniforge3/envs/torch-nightly/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 42


42

In [3]:

augmentations = {'train':
    T.Compose(
    [
        T.Resize((224, 224)),
        T.RandomApply([T.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1)], p=0.8),
        T.RandomGrayscale(p=0.2),
        T.GaussianBlur(kernel_size=9),
        T.ToTensor(),
    ]),
    'val':
    T.Compose(
    [
        T.Resize((224, 224)),
        T.RandomApply([T.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1)], p=0.8),
        T.RandomGrayscale(p=0.2),
        T.GaussianBlur(kernel_size=9),
        T.ToTensor(),
    ])
}


In [4]:
data_path = '/Users/caghankoksal/Desktop/development/Flamingo-playground/physionet.org/files/mimic-cxr/2.0.0/files/'
mimic_datamodule = MIMICCXRDataModule(data_path, transforms=augmentations, only_images=False, batch_size=32,
                                limit_num_samples=None, num_data_workers=0)
train_loader = mimic_datamodule.train_dataloader()
val_loader = mimic_datamodule.val_dataloader()

100%|██████████| 1/1 [00:00<00:00,  1.50it/s]


In [5]:
len(mimic_datamodule.train_dataset)

4355

In [6]:
len(train_loader)

137

In [7]:
137*32

4384

In [8]:
for batch in train_loader:
    print("Image shape : ",batch["image"].shape)
    print(batch["text"][0])# First sample in the batch
    print("Input Ids shape : ",batch["input_ids"].shape)
    print("Targets shape : ",batch["targets"].shape)
    break

Image shape :  torch.Size([32, 3, 224, 224])
<BOS> <image> Output: FINAL REPORT AP CHEST 1:28 A.M.  HISTORY:  -year-old woman with pneumonia and C. difficile infection. IMPRESSION:  AP chest compared to  through : There has been very little change in the appearance of the lungs in this patient with severe pulmonary fibrosis over the course of this hospitalization, probably a function of either concurrent pneumonia or acceleration of pulmonary fibrosis.  Small bilateral pleural effusions have appeared intermittently, greater today than yesterday, suggesting some component of cardiac decompensation.  ET tube is in standard placement.  PIC line ends in the region of the superior cavoatrial junction alongside a right internal jugular line.  Nasogastric tube passes below the diaphragm and out of view.  No pneumothorax. <EOC>
Input Ids shape :  torch.Size([32, 1, 256])
Targets shape :  torch.Size([32, 1, 256])


In [9]:
from src.models.multimodal.flamingo_palm_clip import FlamingoClipPalm
import pytorch_lightning as pl

In [10]:
pretrained_clip_path = '/Users/caghankoksal/Desktop/development/PubMedCLIP_ViT32.pth'

In [11]:
BATCH_SIZE = 32
NUM_EPOCHS = 200
len(mimic_datamodule.train_dataset)//BATCH_SIZE*NUM_EPOCHS

27200

In [12]:
model = FlamingoClipPalm(pretrained_clip_path = pretrained_clip_path,
                        total_steps=len(mimic_datamodule.train_dataset)//BATCH_SIZE*NUM_EPOCHS)

In [13]:
trainer = pl.Trainer(max_epochs=200,deterministic=True,
                     accelerator="cpu", devices=1)
trainer.fit(model, train_dataloaders=train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/caghankoksal/miniforge3/envs/torch-nightly/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:133: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")
Missing logger folder: /Users/caghankoksal/Desktop/SS2022/mlmi-vqa/notebooks/lightning_logs

  | Name          | Type              | Params
----------------------------------------------------
0 | vit_clip      | VisionTransformer | 87.8 M
1 | flamingo_palm | FlamingoPaLM      | 167 M 
----------------------------------------------------
79.7 M    Trainable params
87.8 M    Non-trainable params
167 M     Total params
670.105   Total estimated model params size (MB)
/Users/caghankoksal/miniforge3/envs/torch-nightly/li

Epoch 0:   3%|▎         | 4/137 [00:52<29:13, 13.19s/it, loss=10.6, v_num=0]

/Users/caghankoksal/miniforge3/envs/torch-nightly/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
! pip install pyrsistent